FileDownloader 

In [1]:
# This class is responsible for downloading files from the internet.
import urllib.request
import os

class FileDownloader:
    def __init__(self, url, file_path):
        self.url = url
        self.file_path = file_path

    def download(self):
        if not os.path.exists(self.file_path):
            print(f"Downloading {self.file_path}...")
            urllib.request.urlretrieve(self.url, self.file_path)
            print(f"{self.file_path} downloaded successfully!")


YoloDetector

In [2]:
# This class is responsible for loading the YOLO model and detecting objects in frames.
import cv2

class YoloDetector:
    def __init__(self, weights_path, cfg_path, names_path):
        self.weights_path = weights_path
        self.cfg_path = cfg_path
        self.names_path = names_path
        self.net = cv2.dnn.readNet(weights_path, cfg_path)
        self.layer_names = self.net.getLayerNames()
        self.output_layers = self._get_output_layers()
        self.classes = self._load_classes()

    def _get_output_layers(self):
        try:
            return [self.layer_names[i - 1] for i in self.net.getUnconnectedOutLayers().flatten()]
        except AttributeError:
            return [self.layer_names[i[0] - 1] for i in self.net.getUnconnectedOutLayers()]

    def _load_classes(self):
        with open(self.names_path, "r") as f:
            return f.read().strip().split("\n")

    def detect_objects(self, frame):
        height, width, _ = frame.shape
        blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
        self.net.setInput(blob)
        detections = self.net.forward(self.output_layers)

        max_area = 0
        best_face = None
        best_box = None

        for detection in detections:
            for obj in detection:
                scores = obj[5:]
                class_id = scores.argmax()
                confidence = scores[class_id]

                if confidence > 0.5 and self.classes[class_id] == "person":
                    center_x = int(obj[0] * width)
                    center_y = int(obj[1] * height)
                    box_w = int(obj[2] * width)
                    box_h = int(obj[3] * height)
                    x = int(center_x - box_w / 2)
                    y = int(center_y - box_h / 2)

                    area = box_w * box_h

                    if area > max_area:
                        max_area = area
                        best_face = frame[y:y + box_h, x:x + box_w]
                        best_box = (x, y, box_w, box_h)

        return best_face, best_box


EmotionAnalyzer

In [3]:
# This class is responsible for analyzing emotions from the detected face using DeepFace.
from deepface import DeepFace

class EmotionAnalyzer:
    def __init__(self):
        pass

    def analyze(self, face):
        try:
            result = DeepFace.analyze(face, actions=['emotion'], enforce_detection=False)
            return result[0]['dominant_emotion']
        except Exception as e:
            print(f"Error in emotion detection: {e}")
            return None


VideoProcessor

In [4]:
# This class is responsible for capturing video frames and displaying them.
import cv2

class VideoProcessor:
    def __init__(self, video_source=0):
        self.cap = cv2.VideoCapture(video_source)

    def capture_frame(self):
        ret, frame = self.cap.read()
        return frame

    def release(self):
        self.cap.release()

    def display_frame(self, frame):
        cv2.imshow("YOLO-based Real-time Emotion Detection", frame)


File Downloading for YOLO Files

In [ ]:
# This code downloads the necessary YOLO files (weights, config, and class names) if they are not already present.
# URLs to download the necessary files
yolo_weights_url = "https://github.com/AlexeyAB/darknet/releases/download/yolov4/yolov4-tiny.weights"
yolo_cfg_url = "https://github.com/AlexeyAB/darknet/raw/master/cfg/yolov4-tiny.cfg"
coco_names_url = "https://github.com/AlexeyAB/darknet/raw/master/data/coco.names"

# File paths
weights_path = "yolov4-tiny.weights"
cfg_path = "yolov4-tiny.cfg"
names_path = "coco.names"

# Download YOLO files if not already present
FileDownloader(yolo_weights_url, weights_path).download()
FileDownloader(yolo_cfg_url, cfg_path).download()
FileDownloader(coco_names_url, names_path).download()


Main Execution

In [6]:
# This cell runs the main loop, which captures video, detects faces, analyzes emotions, and displays the results in real-time.
def main():
    # Initialize components
    yolo_detector = YoloDetector(weights_path, cfg_path, names_path)
    emotion_analyzer = EmotionAnalyzer()
    video_processor = VideoProcessor()

    while True:
        frame = video_processor.capture_frame()
        best_face, best_box = yolo_detector.detect_objects(frame)

        if best_face is not None:
            emotion = emotion_analyzer.analyze(best_face)
            if emotion:
                x, y, box_w, box_h = best_box
                cv2.rectangle(frame, (x, y), (x + box_w, y + box_h), (0, 255, 0), 2)
                cv2.putText(frame, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        video_processor.display_frame(frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_processor.release()
    cv2.destroyAllWindows()

# Run the main function
main()


Error in emotion detection: list index out of range
Error in emotion detection: list index out of range
Error in emotion detection: list index out of range
Error in emotion detection: list index out of range
Error in emotion detection: list index out of range
Error in emotion detection: list index out of range
Error in emotion detection: list index out of range
Error in emotion detection: list index out of range
Error in emotion detection: list index out of range
Error in emotion detection: list index out of range
Error in emotion detection: list index out of range
Error in emotion detection: list index out of range
Error in emotion detection: list index out of range
Error in emotion detection: list index out of range
Error in emotion detection: list index out of range
Error in emotion detection: list index out of range
Error in emotion detection: list index out of range
Error in emotion detection: list index out of range
Error in emotion detection: list index out of range
Error in emo